In [ ]:
#5절 
콘텐츠 기반 필터링 실습.

# -장르 속성을 이용한 영화 콘텐츠 기반 필터링
콘텐츠 기반 필터링은, 사용자가 특정 영화를 감상하고 (감상)
그 영화를 좋아했다면 (좋아한다)
그 영화와 비슷한 특성/속성, 구성 요소 등을 가진 다른 영화를 추천하는 것. (비슷하다)

가령 영화 인셉션을 재미있게 봤다면,
인셉션의 장르인 액션, 공상과학으로 높은 평점을 받은 (같은 장르, 높은 평점)
다른 영화를 추천하거나
인셉션의 감독인 크리스토퍼 놀란의 다른 영화를 추천하는 방식. (같은 감독)

이렇게 영화-상품/서비스 간의 유사성을 판단하는 기준이 
영화를 구성하는 다양한 콘텐츠-장르,감독,배우,평점,키워드,영화 설명을 기반으로
하는 방식이 콘텐츠 기반 필터링입니다.

콘텐츠 기반 필터링 추천 시스템을, 영화 장르 속성을 기반으로 만들어 봅시다.
장르 칼럼 값의 유사도를 비교한 뒤,
그중 높은 평점을 가지는 영화를 추천하는 방식.

장르 유사도가 높은 영화 중에서, 평점이 높은 순으로 영화를 추천한다.
#순서 관계. 내가 한건 평균..비슷하게했다

# -데이터 로딩 및 가공

# -장르 콘텐츠 유사도 측정
장르별 유사로를 측정하는 가장 간단한 방법: #더 좋은 다른 방법도 있다는거?
    장르들을 문자열로 변경하고, CountVectorizer로 피처 벡터화한 행렬 데이터 값을
    코사인 유사도로 비교하는 것.

# -장르 콘텐츠 필터링을 이용한 영화 추천
장르 기반으로 추천하는 코드가 일단 완성된 상태.
문제점: 
    장르는 비슷한데, 평점이 낮은 영화를 추천한다.
    그러니까 장르로 필터링 후, 평점도 고려하고 싶다는 거지.
    
영화의 평점에 따라 필터링해서 최종 추천하는 방식으로 변경해봅시다.
여러 관객이 평가한 평점을 평균한 데이터를 이용해봅시다.
그런데 1,2명의 소수 관객이 특정 영화에 만점이나 매우 높은 평점을 부여해
    왜곡된 데이터를 갖고 있습니다.
    sort_values()를 이용해 평점을 출력해 확인해봅시다.
    ex) movies_df[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]
    
횟수가 매우 작은 왜곡된 평점 데이터를 회피할 수 있도록 
    평점에 평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요합니다.
    가중 평점 공식: 평가 횟수에 대한 가중치가 부여된 평점.
    (v/(v+m)) * R + (m/(v+m)) * C
    
    R,C는 평균 평점. 개별/전체
    v의 평가 횟수. 
    
    m은 반영한 횟수-독립변수?
    평가 횟수에 따른 가중치를 직접 조절하는 역할을 한다.
    m값을 높이면, 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 부여합니다.
    
    예제에서는 m값을 전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정합니다.
    상위 60%값은 Serie 객체의 quantile()을 이용해 추출합니다.
    ex) m = movies_df['vote_count'].quantile(0.6)
    
기존 평점을 새로운 가중 평점으로 변경하는 함수를 생성하고,
    이를 이용해 새로운 평점 정보인 vote_weighted 값을 만들겠습니다.
    이 함수는 DataFrame의 레코드를 인자로 받아
    이 레코드의 vote_count와 vote_average 칼럼, 
    그리고 미리 추출된 m과 C값을 적용해, 레코드별 가중 평점을 반환합니다.
    해당 함수를 movies_df의 apply()의 인자로 입력해 가중 평점을 계산합니다.
    ...
    
하지만 장르만으로 영화가 전달하는 많은 요소와 분위기, 그리고 개인이 좋아하는 성향을
    반영하기에는 부족할 수 있습니다.
    아마 좋아하는 영화배우나 감독을 보고 영화를 선택하는 경우가 더 많을 것입니다.
    
    장르를 기반으로 한 콘텐츠 필터링 예제를 좀 더 다양한 콘텐츠 기반으로
    확장할 수 있습니다.
    다른 콘텐츠 기반으로 확장해 추천 시스템을 고도화하는 부분은 숙제로 남겨두고,
    이어서 아이템 기반 최근접 이웃 협업 필터링을 구현해 봅시다.



In [ ]:
#6절
##아이템 기반 최근접 이웃 협업 필터링 실습
최근접 이웃 협업 필터링은 사용자 기반과 아이템 기반으로 분류합니다.
일반적으로 추천 정확도가 더 뛰어난 아이템 기반의 협업 필터링을 구현해 봅시다.

협업 필터링 기반의 영화 추천을 위해서는,
    사용자가 영화의 평점을 매긴 사용자-영화 평점 행렬 데이터 세트가 필요합니다.
    
# -데이터 가공 및 변환
...
협업 필터링은 이 ratings.csv 데이터 세트와 같이 
    사용자와 아이템 간의 평점-또는 다른 유형의 액션에 기반해
    추천하는 시스템입니다.
    (평점은 min-max가 있는 데이터 겠지?
    선호도로 계산하는 방법의 문제점은 max가 없다는 것
    어떻게 max를 부여할 수 있을까? ex. 어떤 점수든 1+3+5로 나눠서 임시 평균을 계산한다거나?
    그럴 경우의 문제점은 뭐가 있을까?)
    
먼저 로우 레벨 형태의 원본 데이터 세트를
모든 사용자를 로우로, 모든 영화를 칼럼으로 구성한 데이터 세트로 변경해야 합니다.
    이 같은 변환은 DataFrame의 pivot_table()을 이용하면 쉽게 할 수 있습니다.
    로우 레벨의 값을 칼럼으로 변경하는 데 효과적입니다.
    
    pivot_table()에 인자로 columns='movieId'와 같이 부여하면
    movieId 칼럼의 모든 값이 새로운 칼럼 이름으로 변환됩니다.
    
    ratings.pivot_table('rating', index='userId', columns='movieId')와 같이
    호출하면 로우 레벨은 userId, 칼럼은 모두 movieId 칼럼이 있는 값으로
    칼럼 이름이 바뀌고, 데이터는 rating 칼럼이 있는 값이 할당됩니다.
    ...
    
    NaN값이 많은 이유는 사용자가 평점을 매기지 않은 영화가
    칼럼으로 변환되면서 NaN값으로 값이 할당됐기 때문입니다.
    최소 평점이 0.5이므로 NaN은 모두 0으로 변환하겠습니다.
    
    그 전에 먼저, 칼럼명이 현재 movieId 숫자 값과 같이 할당되어 있어
    사용자가 평점을 준 영화가 어떤 영화인지 알기 어렵습니다.
    
    가독성을 높이기 위해 칼럼명을 movieId가 아닌 영화명 title로 변경하겠습니다.
    영화명은 ratings에 존재하지 않고 movies 데이터 세트에 존재합니다.
    
    ratings와 movies를 조인해, title 칼럼을 가져온 뒤에
    pivot_table()의 인자로 columns에 movieId가 아닌 titled릉 비력해
    title로 피벗하겠습니다.
    이후에 NaN을 0으로 변환합니다.
    ...

# -영화 간 유사도 산출
변환된 사용자-영화 평점 행렬 데이터 세트를 이용해
    영화 간의 유사도를 측정하겠습니다.
    영화 같의 유사도는 코사인 유사도를 기반으로 하고
    사이킷런의 cosine_similarity()를 이용해 측정합니다.
    
    만든 ratings_matrix 데이터 세트에 cosine_similarity()를 적용하면
    영화간 유사도를 측정할 수 없습니다.
    cosine_similarity()는 행을 기준으로 서로 다른 행을 비교해 유사도를 산출합니다.
    그런데 ratings_matrix는 userId가 기준인 행 레벨 데이터이므로
    여기서 cosine_similarity()를 적용하면 영화 같의 유사도가 아닌
    사용자 간의 유사도가 만들어집니다.
    
    영화를 기준으로 cosine_similarity()를 적용하려면
    현재의 ratings_matrix가 행 기준이 영화가 되고 열기준이 사용자가 되어야 합니다.
    
    그렇게 하려면 ratings_matrix 데이터의 행과 열의 위치를 변경하면 되는데,
    판다스는 이 같은 전치 행렬 변경을 위해 transpose() 함수를 제공합니다.
    ratings_matrix에 transpose()를 적용해 행과 열을 서로 바꾼 새로운 행렬을 만들 수 있습니다.
    ...
    
좀 더 직관적인 영화의 유사도 값을 표현하기 위해
cosine_similarity()로 반환된 넘파이 행렬에 영화명을 매핑해
DataFrame으로 변환하겠습니다.
    ...
    
만들어진 아이템 기반 유사도 데이터는,
    사용자의 평점 정보를 모두 취합해 
    영화에 따라 유사한 다른 영화를 추천할 수 있게 해줍니다.
    
이번에는 이 아이템 기반 유사도 데이터를 이용해
    개인에게 특화된Personalized 영화 추천 알고리즘을 만들어 보겠습니다.


# -아이템 기반 최근접 이웃 협업 필터링으로 개인화된 영화 추천
(위보다 이게 더 좋은 방식인듯)
앞 예제에서 만든 아이템 기반의 영화 유사도 데이터는
    모든 사용자의 평점을 기준으로 영화의 유사도를 생성했고,
    이를 이용해 영화를 추천합니다.
    
    하지만 이는 개인적인 취향을 반영하지 않고
    영화 간의 유사도만을 가지고 추천한 것입니다.
    (그래서 개인적인 취향은 어떻게 반영하는가)
    
    이번 절에서는, 영화 유사도 데이터를 이용해
    최근접 이웃 협업 필터링으로 개인에게 최적화된 영화 추천을 구현해 보겠습니다.
    
    개인화된 영화 추천의 가장 큰 특징은
    개인이 아직 관람하지 않은 영화를 추천한다는 것입니다.
    
    아직 관람하지 않은 영화에 대해서 
    아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기반으로
    새롭게 모든 영화의 예측 평점을 계산한 후
    높은 예측 평점을 가진 영화를 추천하는 방식입니다.
    
    아이템 기반의 협업 필터링에서 개인화된 예측 평점 공식.
    사용자u, 아이템i의 개인화된 예측 평점 값 =
        아이템i와 가장 유사도가 높은 top-N개 아이템의 유사도 벡터 (코사인 유사도?)
        * 사용자u의 아이템i와 가장 유사도가 높은 top-N개 아이템에 대한 실제 평점 벡터
        
    그러니까 아이템 (유사도 벡터*평점 벡터)합을,
        유사도 벡터 합 으로 나눈다는 의미.
        
    여기서 N 값은 아이템의 최근접 이웃 범위 계수를 의미한다.
    이는 특정 아이템과 유사도가 가장 높은 top-N개의 다른 아이템을 추출하는데 사용된다.
    
    먼저 N의 범위에 제약을 두지 않고
    모든 아이템으로 가정하고 예측 평점을 구하는 로직을 작성한 뒤에
    top-N 아이템을 기반으로 협업 필터링을 수행하는 로직으로 변경하겠습니다.
    
영화 간의 유사도를 가지는 DataFrame인 item_sim_df와
    사용자-영화 평점 DataFrame인 ratings_matrix 변수를 계속 활용해,
    사용자별로 최적화된 평점 스코어를 예측하는 함수를 만들겠습니다.
    
    함수명은 predict_rating()이며, 
    인자로 사용자-영화 평점 넘파이 행렬과
    영화 간의 유사도를 가지는 넘파이 행렬을 입력받고
    
    이를 이용해 개인화된 예측 평점을 계산합니다.
    
N의 범위에 제약을 두지 않는다면,
    사용자별 영화 예측 평점은 
    
    사용자u의 모든 영화에 대한 실제 평점과  #모든 영화에 대한 해당 사용자의 평점
    
    영화i의 다른 모든 영화와의 코사인 유사도를 #모든 영화의 코사인 유사도
    벡터 내적 곱한 값을 
    
    정규화를 위해 ..합으로 나눈 것을 의미합니다. #유사도 합
    
    #왜? 사용자의 실제 평점이 왜 필요한거?
    #이전 예제에서는 사용자 협업 필터링이었다..
    #사용자의 코사인 유사도*아이템에 대한 그사용자평점
    
    ...

예측 평점이 사용자별 영화의 실제 평점과
    영화의 코사인 유사도를 내적한 값이기 때문에
    기존에 영화를 관람하지 않아 0에 해당했던 실제 영화 평점이
    예측에서는 값이 부여되는 경우가 많이 발생합니다.
    
    (우리도 선호도를 0으로 두고 계산하면, 이런 문제?가 발생하겠군)
    
    예측 평점이 실제 평점에 비해 작을 수 있습니다.
    이는 내적 결과를 코사인 유사도 벡터 합으로 나누었기 때문에 생기는 현상입니다.
    
이 예측 결과가 원래의 실제 평점과 얼마나 차이가 있는지 확인해 보겠습니다. #평가?
    예측 평가 지표는 MSE를 적용하겠습니다.
    
    MSE를 측정할 때 유의할 점이 있습니다.
    사용자가 영화의 평점을 주지 않은 경우,
    앞에서는 평점을 0으로 부과했습니다.
    
    하지만 앞에서 개인화된 예측 점수는 평점을 주지 않은 영화에 대해서도
    아이템 유사도에 기반해 평점을 예측했습니다.
    따라서 실제와 예측 평점의 차이는
    기존에 평점이 부여된 데이터에 대해서만 오차 정도를 측정하겠습니다.
    예측 평가 지표인 MSE를 계산하는 get_mse() 함수를 만들고 결과를 확인합시다.
    ...
    
실제 값과 예측값은 서로 스케일이 다르기 때문에 MSE가 클 수도 있습니다.
중요한 것은 MSE를 감소시키는 방향으로 개선하는 것입니다.

predict_rating()은 사용자별 영화의 예측 평점을 계산하기 위해
    해당 영화와 다른 모든 영화 간의 유사도 벡터를 적용합니다.
    
    많은 영화의 유사도 벡터를 이용하다 보니
    상대적으로 평점 예측이 떨어졌습니다.
    (많은 영화를 대상으로 해서 그런거라고?)
    
    특정 영화와 가장 비슷한 유사도를 가지는 영화에 대해서만 
    유사도 벡터를 적용하는 함수로 변경해 봅시다.
    (어떻게 가장 비슷하다고 판단하는 건데?)
    
    predict_rating_topsim()은 predict_rating()과 유사하지만
    N인자를 가지고 있어 top-N 유사도를 가지는 영화 유사도 벡터만
    예측값을 계산하는 데 적용합니다.
    (사실 우리 서비스에 이 부분에 대한 내용은 걱정하지 않아도 될 것 같다)
    (사용자 기반 협업 필터링이라면 모를까)
    
    아쉬운 점은 이러한 계산을 위해서는
    개별 예측값을 구하기 위해 행,열 별로 for루프를 수행하면서
    top-N 유사도 벡터를 계산해야 하기 때문에 수행시간이 오래 걸린다는 점입니다.
    데이터의 크기가 매우 커지면 오래 걸리는 로직.
    ...
    
    

In [ ]:
#7절
이번에는 행렬 분해를 이용한 잠재 요인 협업 필터링을 구현해 봅시다.
행렬 분해 잠재 ....


In [ ]:
#추천 시나리오 4가지
#5절이었나? 순차적 필터링
#아이템, 사용자, 잠재-모델

그리고 기존 데이터 말인데..
만약에 많은 식당을 대상으로 한다면
맛집 게시판 크롤링하자

크롤링할 수 있는 다른 곳도 살펴보던지

In [ ]:
#웨이크업 굿나잇 아이템 기반 최근접 이웃 협업 필터링 해본다면

예전에 생각해둔 선호도를 기반으로 하면 되지 않을까 싶다.
초반에 데이터가 별로 없을 때는 콘텐츠 기반으로 일일이 필터링하고,
데이터가 좀 쌓였다 싶으면 테스트 시작하는 방식으로?

협업 필터링은 이 ratings.csv 데이터 세트와 같이 
    사용자와 아이템 간의 평점-또는 다른 유형의 액션에 기반해
    추천하는 시스템입니다.
    (평점은 min-max가 있는 데이터 겠지?
    선호도로 계산하는 방법의 문제점은 max가 없다는 것
    어떻게 max를 부여할 수 있을까? ex. 어떤 점수든 1+3+5로 나눠서 임시 평균을 계산한다거나?
    그럴 경우의 문제점은 뭐가 있을까?)
    
    아이템 기반의 협업 필터링에서 개인화된 예측 평점 공식.        
    그러니까 아이템 (유사도 벡터*평점 벡터)합을,
        유사도 벡터 합 으로 나눈다는 의미.
        
선호도를 정규화하려면 평점에 횟수를 고려하듯.
    가중 평점 처럼 계산해야 할듯

In [ ]:
# 웨이크업 시나리오, 단게를 넘어가는 부분 잘 모르겠으니까
# 테스트는 모든 단계 다 출력해서 눈으로 확인해보기
처음에 데이터가 일도 없을 때는,
    고유 아이템 재료,가격 등등 기반으로 추천 제공
    
좀 데이터가 있으면 콘텐츠 기반 5절

더 쌓이면 6절

In [ ]:
#웨이크업 굿나잇 메뉴를 콘텐츠 기반 필터링 해본다면
#데이터셋 구성은 어떻게 될지 생각해보자.

1메뉴이름,2가격,3평점,4구매횟수=투표수,5재료=키워드

+날씨, 감정: 구매횟수나 평점처럼 집계하는 방법 생각해보기
    
장르 유사도 반영 후, 평점 고려.
말고 통합적으로 고려하는 방법 없나?
여기서 통합적이 아주 주관적이다
각 코사인 유사도 행렬에 각각 다른 가중치를 부여할 수 있겠다
가중치 부여 기준은 뭘까?

**가중 평점 공식: 평가 횟수에 대한 가중치가 부여된 평점.
응용하면 다른 구성요소에도 적용해볼 수 있겠지? 582

기준 영화 인덱스 제외 방법 있다.
2배수에 해당하는 장르 기반 후보군에서 가중평점으로 1배수 추출한다.
(근데 메뉴가 엄청 많은게 아니라 우리는 전체 루프 돌면 되겠다)

각 코사인 유사도를 전체 더할 것인가,
가중 합계할 것인가,
순차 필터링을 할 것인가..
(사용자 성향 분석해서 개인화하면 되게 좋을것같다.)